In [30]:
import code

import numpy as np
import gurobipy as gp

model = gp.Model("polynomial_program")

# build the monomial vectors
n = 1
k = 6

monomials = code.tensors.get_monomial_indices(n, k)

# create the vars
x = model.addVars(len(monomials), name='x')

# get the constraints
A = code.tensors.get_moment_operator(
    n=n, 
    k_half=k // 2, 
    monomials=monomials,
    localizer=None
)

z = model.addVars(*A.shape[1:3], name='z')
Ax = gp.quicksum(A[i] * x[i] for i in range(A.shape[0]))

# set equality of Ax with z, our semidefinite vector
for i in range(A.shape[1]):
    for j in range(A.shape[2]):
        model.addConstr(Ax[i,j] == z[i,j], name=f'Ax_({i},{j})=z_({i},{j})')